In [3]:
import pandas as pd
import requests
import numpy as np


In [ ]:
import pandas as pd

In [27]:

df = pd.read_csv('mlb_plays.csv')

In [28]:
df = df.tail(10000)

In [29]:
df.head()

,gamePk,gameDate,inning,halfInning,playDescription,eventType,batter,pitcher,batSide,pitchHand,...,balls,strikes,pitchType,pitchSpeed,strikeZoneTop,strikeZoneBottom,pitchResult,isInPlay,isStrike,isBall
6093403,716382,2023-09-30T02:15:00Z,6,top,"Jason Heyward grounds out, pitcher Sean Hjelle...",field_out,Jason Heyward,Sean Hjelle,Left,Right,...,3,2,Knuckle Curve,84.1,3.560000,1.690000,Swinging Strike,False,True,False
6093404,716382,2023-09-30T02:15:00Z,6,top,"Jason Heyward grounds out, pitcher Sean Hjelle...",field_out,Jason Heyward,Sean Hjelle,Left,Right,...,3,2,Knuckle Curve,84.7,3.586901,1.855326,Ball,False,False,True
6093405,716382,2023-09-30T02:15:00Z,6,top,"Jason Heyward grounds out, pitcher Sean Hjelle...",field_out,Jason Heyward,Sean Hjelle,Left,Right,...,3,2,Knuckle Curve,83.8,3.515614,1.712222,Ball,False,False,True
6093406,716382,2023-09-30T02:15:00Z,6,top,"Jason Heyward grounds out, pitcher Sean Hjelle...",field_out,Jason Heyward,Sean Hjelle,Left,Right,...,3,2,Sinker,95.5,3.560000,1.690000,Foul,False,True,False
6093407,716382,2023-09-30T02:15:00Z,6,top,"Jason Heyward grounds out, pitcher Sean Hjelle...",field_out,Jason Heyward,Sean Hjelle,Left,Right,...,3,2,Sinker,96.2,3.560000,1.690000,"In play, out(s)",True,False,False


In [7]:
print(df.dtypes)

gamePk                int64
gameDate             object
inning                int64
halfInning           object
playDescription      object
eventType            object
batter               object
pitcher              object
batSide              object
pitchHand            object
outs                  int64
balls                 int64
strikes               int64
pitchType            object
pitchSpeed          float64
strikeZoneTop       float64
strikeZoneBottom    float64
pitchResult          object
isInPlay               bool
isStrike               bool
isBall                 bool
dtype: object


DROP THESE. We DO NOT NEED THESE

In [30]:
df.drop('isInPlay',axis=1,inplace=True)
df.drop('isBall',axis=1,inplace=True)
df.drop('isStrike',axis=1,inplace=True)
df.drop('playDescription',axis=1,inplace=True)
df.drop('pitchResult',axis=1,inplace=True)
df.drop('gameDate',axis=1,inplace=True)

In [31]:
print(df.dtypes)

gamePk                int64
inning                int64
halfInning           object
eventType            object
batter               object
pitcher              object
batSide              object
pitchHand            object
outs                  int64
balls                 int64
strikes               int64
pitchType            object
pitchSpeed          float64
strikeZoneTop       float64
strikeZoneBottom    float64
dtype: object


In [71]:
for players in df:
    print(players)

gamePk
gameDate
inning
halfInning
eventType
batter
pitcher
batSide
pitchHand
outs
balls
strikes
pitchType
pitchSpeed
strikeZoneTop
strikeZoneBottom


In [61]:
#ignore
#new_df = df.head(1000)
#new_df.to_csv('smaller_dataset.csv',index=False)

From the play_by_play dataset we find the name from both batter and pitcher.

then for each name we find the ID


In [32]:
def get_player_id(name):
    url = f"https://statsapi.mlb.com/api/v1/people/search?names={name}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
    try:
        date = data['people'][0]["lastPlayedDate"].split("-")[0]
    except KeyError:
        date = '2024'

    return(data['people'][0]['id'],date)

In [33]:
get_player_id('Aaron Judge')

(592450, '2024')

In [22]:
url = f"https://statsapi.mlb.com/api/v1/people/{624577}"
response = requests.get(url)

if response.status_code != 200:
   print('nah')

data = response.json()
data['people'][0] # First player entr

{'id': 624577,
 'fullName': 'Yasiel Puig',
 'link': '/api/v1/people/624577',
 'firstName': 'Yasiel',
 'lastName': 'Puig',
 'primaryNumber': '66',
 'birthDate': '1990-12-07',
 'currentAge': 34,
 'birthCity': 'Cienfuegos',
 'birthCountry': 'Cuba',
 'height': '6\' 2"',
 'weight': 240,
 'active': True,
 'primaryPosition': {'code': '9',
  'name': 'Outfielder',
  'type': 'Outfielder',
  'abbreviation': 'RF'},
 'useName': 'Yasiel',
 'useLastName': 'Puig',
 'boxscoreName': 'Puig',
 'nickName': 'Wild Horse',
 'gender': 'M',
 'nameMatrilineal': 'Valdes',
 'isPlayer': True,
 'isVerified': True,
 'pronunciation': 'yah-SEE-el PWEEG',
 'mlbDebutDate': '2013-06-03',
 'batSide': {'code': 'R', 'description': 'Right'},
 'pitchHand': {'code': 'R', 'description': 'Right'},
 'nameFirstLast': 'Yasiel Puig',
 'nameSlug': 'yasiel-puig-624577',
 'firstLastName': 'Yasiel Puig',
 'lastFirstName': 'Puig, Yasiel',
 'lastInitName': 'Puig, Y',
 'initLastName': 'Y Puig',
 'fullFMLName': 'Yasiel Puig',
 'fullLFMName':

THe we find the stats.

In [34]:
def get_stats(id,year):
    url = f"https://statsapi.mlb.com/api/v1/people/{id}/stats?stats=season&season={year}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
    else:
        return None
    try:
        return data['stats'][0]["splits"][0]["stat"]
    except IndexError:
        print(id)
        return None

id,date = get_player_id('Jon Lester')
get_stats(id,date)

{'gamesPlayed': 28,
 'gamesStarted': 28,
 'groundOuts': 173,
 'airOuts': 146,
 'runs': 84,
 'doubles': 33,
 'triples': 0,
 'homeRuns': 25,
 'strikeOuts': 91,
 'baseOnBalls': 55,
 'intentionalWalks': 5,
 'hits': 159,
 'hitByPitch': 3,
 'avg': '.285',
 'atBats': 558,
 'obp': '.349',
 'slg': '.478',
 'ops': '.827',
 'caughtStealing': 5,
 'stolenBases': 11,
 'stolenBasePercentage': '.688',
 'groundIntoDoublePlay': 14,
 'numberOfPitches': 2380,
 'era': '4.71',
 'inningsPitched': '141.1',
 'wins': 7,
 'losses': 6,
 'saves': 0,
 'saveOpportunities': 0,
 'holds': 0,
 'blownSaves': 0,
 'earnedRuns': 74,
 'whip': '1.51',
 'battersFaced': 627,
 'outs': 424,
 'gamesPitched': 28,
 'completeGames': 0,
 'shutouts': 0,
 'strikes': 1453,
 'strikePercentage': '.610',
 'hitBatsmen': 3,
 'balks': 0,
 'wildPitches': 0,
 'pickoffs': 0,
 'totalBases': 267,
 'groundOutsToAirouts': '1.18',
 'winPercentage': '.538',
 'pitchesPerInning': '16.84',
 'gamesFinished': 0,
 'strikeoutWalkRatio': '1.65',
 'strikeoutsPe

Next I want to add relevant stats, LIKE:
1. Batter Stats:
Batting Average (AVG): Measures how well the batter gets on base.
On-base Percentage (OBP): Indicates how frequently the batter reaches base.
Slugging Percentage (SLG): Measures the power of the batter (extra-base hits).
On-base Plus Slugging (OPS): Combination of OBP and SLG; overall offensive effectiveness.
Home Runs (HR): Shows power and run-driving capability.
Runs Batted In (RBI): How many runs the batter has driven in.
Strikeouts (SO): How often the batter strikes out.
Walks (BB): How many times the batter walked.
Total Bases (TB): Total of all bases the batter has gained via hits.
Stolen Bases (SB): Ability to steal bases.
Batting Streak (if available): A streak of consecutive games with a hit.
2. Pitcher Stats:
Earned Run Average (ERA): How many earned runs a pitcher gives up per 9 innings.
Walks and Hits per Inning Pitched (WHIP): Total walks and hits given up per inning.
Strikeouts (SO): How often the pitcher strikes out batters.
Batting Average Against (BAA): The average batting average of hitters against the pitcher.
Wins (W): Number of games the pitcher has won.
Losses (L): Number of games the pitcher has lost.
Saves (SV): Number of games in which the pitcher finished a win.
Strikeout-to-Walk Ratio (K/BB): Measures the pitcher's control and ability to miss bats.
Ground Outs to Air Outs (GO/AO): Shows how many batters are grounded out versus fly outs.
Inherited Runners Scored: Runners left on base that scored after the pitcher entered the game.


if there anre'nt some stats skip those

find relevant stats for batter and pitcher sperately 
Then add directly to the play_by_play dataframe.
Add suffixes if you want(for clarity about which stats belong to which but the ml model wont require it)
Save the final result.

 'avg': '.285',
 'atBats': 558,
 'obp': '.349',
 'slg': '.478',
 'ops': '.827',
 'stolenBasePercentage': '.688',



#pitcher
'era': '4.71',

 'whip': '1.51',

 'outs': 424,

 'strikePercentage': '.610',


 'groundOutsToAirouts': '1.18',

 'strikeoutWalkRatio': '1.65',
 'homeRunsPer9': '1.59',

avg =[]
atBats = []
obp = []
slg =[]
ops =[]
stolenbasepercent = []

era=[]
whip =[]
outs =[]
strikepercent =[]
groundToAirOut =[]
strikeoutWalkRatio =[]
homeRunsPer9 = []



def add_stats_to_lists(stats, is_pitcher=False):
    """Extracts relevant stats from a stats dictionary and appends them to the global lists."""
    global avg, atBats, obp, slg, ops, stolenbasepercent
    global era, whip, outs, strikepercent, groundToAirOut, strikeoutWalkRatio, homeRunsPer9

    if not stats:  # Handles None or empty dictionary
        if is_pitcher:
            era.append(None)
            whip.append(None)
            outs.append(None)
            strikepercent.append(None)
            groundToAirOut.append(None)
            strikeoutWalkRatio.append(None)
            homeRunsPer9.append(None)
        else:
            avg.append(None)
            atBats.append(None)
            obp.append(None)
            slg.append(None)
            ops.append(None)
            stolenbasepercent.append(None)
        return  # Exit early to prevent .get() errors
    
    if is_pitcher:
        era.append(stats.get('era', None))
        whip.append(stats.get('whip', None))
        outs.append(stats.get('outs', None))
        strikepercent.append(stats.get('strikePercentage', None))
        groundToAirOut.append(stats.get('groundOutsToAirouts', None))
        strikeoutWalkRatio.append(stats.get('strikeoutWalkRatio', None))
        homeRunsPer9.append(stats.get('homeRunsPer9', None))
    else:
        avg.append(stats.get('avg', None))
        atBats.append(stats.get('atBats', None))
        obp.append(stats.get('obp', None))
        slg.append(stats.get('slg', None))
        ops.append(stats.get('ops', None))
        stolenbasepercent.append(stats.get('stolenBasePercentage', None))


for index, row in df.iterrows():
    batter = row['batter']
    pitcher = row['pitcher']


    #batter
    id,date = get_player_id(batter)
    stats = get_stats(id,date)

    
    add_stats_to_lists(stats)

    id,date = get_player_id(pitcher)
    stats = get_stats(id,date)
    add_stats_to_lists(stats,True)




In [35]:
import concurrent.futures

def process_row(row):
    batter, pitcher = row['batter'], row['pitcher']

    # Fetch stats in parallel
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_batter = executor.submit(get_stats_for_player, batter, is_pitcher=False)
        future_pitcher = executor.submit(get_stats_for_player, pitcher, is_pitcher=True)

        batter_stats = future_batter.result()
        pitcher_stats = future_pitcher.result()

    return pd.Series(batter_stats + pitcher_stats)  # Combine both stats into one row

def get_stats_for_player(player, is_pitcher):
    id, date = get_player_id(player)
    stats = get_stats(id, date)
    
    if not stats:  # If stats are missing, return None for all fields
        return [None] * (7 if is_pitcher else 6)

    if is_pitcher:
        return [
            stats.get('era', None), stats.get('whip', None), stats.get('outs', None),
            stats.get('strikePercentage', None), stats.get('groundOutsToAirouts', None),
            stats.get('strikeoutWalkRatio', None), stats.get('homeRunsPer9', None)
        ]
    else:
        return [
            stats.get('avg', None), stats.get('atBats', None), stats.get('obp', None),
            stats.get('slg', None), stats.get('ops', None), stats.get('stolenBasePercentage', None)
        ]

# Apply function to all rows using multiprocessing
new_columns = ['b_avg', 'b_atBats', 'b_obp', 'b_slg', 'b_ops', 'b_stolenbasepercent',
               'p_era', 'p_whip', 'p_outs', 'p_strikepercent', 'p_groundToAirOut', 
               'p_strikeoutWalkRatio', 'p_homeRunsPer9']

df[new_columns] = df.apply(process_row, axis=1)

print("✅ Stats added successfully!")


543939
543939
543939
543939
674681
674681
674681
674681
674681
677595
674681
677595
677595
674681
677595
674681
674681
674681
674681
674681
674681
674681
674681
674681
674681
674681
674681
674681
674681
674681
674681
674681
674681
674681
674681
674681
674681
674681
674681
674681
674681
674681
674681
641796
641796
674681
674681
641796
641796
674681
674681
641796
674681
641796
674681
674681
674681
674681
674681
674681
674681
677595
674681
674681
674681
674681
674681
674681
674681
674681
674681
674681
674681
674681
674681
674681
674681
674681
641796
674681
641796
674681
641796
674681
641796
674681
674681
674681
674681
677595
674681
677595
674681
674681
677595
674681
677595
674681
677595
677595
674681
674681
677595
674681
674681
674681
674681
674681
674681
674681
674681
641796
641796
641796
641796
641796
677595
677595
677595
677595
641796
641796
641796
641796
641796
677595
677595
677595
677595
677595
595956
595956
595956
595956
595956
595956
595956
595956
595956
595956
595956
595956
595956

In [37]:
def fill_missing_with_averages(df):
    """Fills missing values in numeric stat columns with their column mean while keeping categorical data untouched."""

    # Define which columns are numerical stats (update as needed)
    numeric_columns = ["p_strikeoutWalkRatio", "b_stolenbasepercent"]  

    # Convert only those columns to numeric, leaving others untouched
    df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

    # Identify **all numeric columns** (actual stats, not categorical ones)
    numeric_stats = df.select_dtypes(include=[np.number]).columns

    # Compute column-wise averages, ignoring NaN values
    column_means = df[numeric_stats].mean(skipna=True)

    # Fill NaN values **only in numeric stat columns**
    df[numeric_stats] = df[numeric_stats].fillna(column_means)

    return df

df = fill_missing_with_averages(df)

In [38]:
df.head(10)

         gamePk  inning halfInning  eventType             batter      pitcher  \
6093403  716382       6        top  field_out      Jason Heyward  Sean Hjelle   
6093404  716382       6        top  field_out      Jason Heyward  Sean Hjelle   
6093405  716382       6        top  field_out      Jason Heyward  Sean Hjelle   
6093406  716382       6        top  field_out      Jason Heyward  Sean Hjelle   
6093407  716382       6        top  field_out      Jason Heyward  Sean Hjelle   
...         ...     ...        ...        ...                ...          ...   
6093498  716382       9        top  field_out      David Peralta  Sean Hjelle   
6093499  716382       9        top  field_out      David Peralta  Sean Hjelle   
6093500  716382       9        top  field_out      David Peralta  Sean Hjelle   
6093501  716382       9        top  field_out      David Peralta  Sean Hjelle   
6093502  716382       9        top  field_out  Enrique Hernández  Sean Hjelle   

        batSide pitchHand  

In [39]:
df.to_csv('new_medium_data.csv',index=False)

In [ ]:
df.head()